#  Práctica Obligatoria - Parte III - CNN

***<p style="text-align:center;">Aprendizaje Automático II</p>***
***<p style="text-align:center;">CNNs</p>***

En esta parte, entrenarás varias redes CNNs y analizarás sus resultados en el dataset GTS. 


### Evaluación - 4/10 puntos

Puntuación de cada parte sobre el total de la práctica:
- **[Ejercicio 1]** 0.25 puntos.
- **[Ejercicio 2]** 0.25 puntos.
- **[Ejercicio 3]** 0.5 puntos.
- **[Ejercicio 4]** 3 puntos.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import transforms

from gts_dataset import GTS


## Importa las funciones que has creado desde `utils.py`:

In [ ]:
from utils import count_trainable_params, train, evaluate, train_and_evaluate, save_full_model, load_full_model, plot_loss_accuracy, plot_confusion_matrix, plot_error_per_class

## Carga el dataset GTS  y crea los dataloaders:

In [ ]:
# Definir las transformaciones y conversión a tensor
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = GTS(csv_file='train.csv', root_dir='./data', transform=transform)
valid_dataset = GTS(csv_file='valid.csv', root_dir='./data', transform=transform)
test_dataset = GTS(csv_file='test.csv', root_dir='./data', transform=transform)

device = 'cpu'
batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

## **[Ejercicio 1]** Crea un modelo `CNN` con las siguientes características:

1. Tres bloques convolucionales (conv2d -> batch norm -> ReLU).
2. Flatten para aplanar características.
3. Una capa densa para clasificación.
4. Comprueba el número de parámetros entrenables y busca una configuración que no supere los **50.000** parámetros (aproximadamente 4 veces menos que la MLP).

In [ ]:
class CNN(nn.Module):
    def __init__(self, ouput_dim):
        super(CNN, self).__init__()

        # Primer bloque convolucional
       
        # Segundo bloque convolucional
       
        # Tercer bloque convolucional

        # Capa de aplanado

        # Linear para la clasificación
        
    def forward(self, x):
        # Primer bloque convolucional
       

        # Segundo bloque convolucional
      

        # Tercer bloque convolucional
      

        # Aplanar la salida de la última capa convolucional
      
        # Linear para clasificación

        return x

count_trainable_params(CNN(ouput_dim=43))

Guarda el modelo en `models.py`

### Entrena el modelo CNN con las funciones que has importado y analiza los resultados:

In [ ]:
# Instancia el modelo
model_cnn = 

# Instancia la función de pérdida y el optimizador
loss_fn = 
optimizer = 

# Fija 10 épocas
num_epochs = 10

# Entrena y evalua el modelo. Usa name='cnn'
train_loss_df, val_loss_df, test_loss, test_accuracy = 




# Guarda el modelo con file_name='cnn.pth'


# Carga los datos y muéstralos
train_loss_file = 'train_loss_cnn.csv'  
valid_loss_file = 'valid_loss_cnn.csv'
train_accuracy_file = 'train_accuracy_cnn.csv'
valid_accuracy_file = 'valid_accuracy_cnn.csv'

plot_loss_accuracy(train_loss_file, valid_loss_file, train_accuracy_file, valid_accuracy_file)

# Calcula la matriz de confusión
cm = plot_confusion_matrix(model_cnn, test_loader, device=device)

# Muestra el porcentaje de error por clase
plot_error_per_class(cm)

Describe las diferencias en los resultados que has obtenido.

############## COMPLETAR ##############

############ Fin COMPLETAR ############

## **[Ejercicio 2]** Dataset permutado: re-entrena la MLP y la CNN

Esta vez, vuelve a entrenar la `MLP`y la `CNN` que has configurado pero con el mismo dataset modificado.

Esta modificación consiste en permutar los píxeles de las imágenes siempre de la misma manera.


In [ ]:
def visualize_subset(dataset, num_images=6):
    """
    Visualiza ejemplos directamente de un Subset o Dataset.
    
    :param dataset: Dataset o Subset que contiene las imágenes y etiquetas.
    :param num_images: Número de imágenes a mostrar.
    """
    indices = np.arange(num_images)
    images = []
    labels = []
    
    for idx in indices:
        image, label = dataset[idx]
        images.append(image)
        labels.append(label)
    
    images = torch.stack(images)  # Convertir la lista de imágenes en un tensor
    images = images.permute(0, 2, 3, 1).numpy()  


    # Mostrar imágenes
    fig, axes = plt.subplots(1, num_images, figsize=(15, 5))
    for idx in range(num_images):
        axes[idx].imshow(images[idx])
        axes[idx].set_title(f"Clase: {labels[idx]}")
        axes[idx].axis('off')
    plt.tight_layout()
    plt.show()


# Visualizar ejemplos del conjunto de entrenamiento
visualize_subset(train_dataset)

In [ ]:
class ShufflePixels:
    def __init__(self, seed=42):
        """
        Inicializa la transformación de shuffle de píxeles.
        Args:
            seed (int): Semilla para generar una permutación fija.
        """
        self.seed = seed
        torch.manual_seed(self.seed)  # Fijar la semilla
        self.permutation = None

    def __call__(self, img):
        """
        Aplica la permutación a los píxeles de la imagen.
        Args:
            img (torch.Tensor): Imagen en formato (C, H, W).
        Returns:
            torch.Tensor: Imagen con píxeles permutados.
        """
        C, H, W = img.shape  # Extraer dimensiones de la imagen

        if self.permutation is None:
            # Generar permutación fija
            self.permutation = torch.randperm(H * W)  # Permutación fija en 1D

        # Aplanar la imagen, permutar y restaurar forma original
        img = img.view(C, H * W)  # (C, H * W)
        img = img[:, self.permutation]  # Aplicar la permutación
        img = img.view(C, H, W)  # Restaurar la forma original
        return img

# Definir las transformaciones con shuffle de píxeles
transform = transforms.Compose([
    transforms.ToTensor(),
    ShufflePixels(seed=42)  # Aplicar shuffle fijo
])
train_dataset_shuffle = GTS(csv_file='train.csv', root_dir='./data', transform=transform)
valid_dataset_shuffle = GTS(csv_file='valid.csv', root_dir='./data', transform=transform)
test_dataset_shuffle = GTS(csv_file='test.csv', root_dir='./data', transform=transform)

device = 'cpu'
batch_size = 16

train_loader_shuffle = DataLoader(train_dataset_shuffle, batch_size=batch_size, shuffle=True, num_workers=4)
valid_loader_shuffle = DataLoader(valid_dataset_shuffle, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader_shuffle = DataLoader(test_dataset_shuffle, batch_size=batch_size, shuffle=False, num_workers=4)


# Visualizar ejemplos del conjunto de entrenamiento
visualize_subset(train_dataset_shuffle)

### Entrena la MLP con:

* `train_loader_shuffle`
* `valid_loader_shuffle`
* `test_loader_shuffle`

Guarda el modelo como `mlp_shuffle.pth` y los resultados de `loss` y `accuracy`con la misma extensión:

In [ ]:
from models import MLP

# Instancia el modelo
model_mlp = 

# Instancia la función de pérdida y el optimizador
loss_fn = 
optimizer =

# Fija 10 épocas
num_epochs = 10

# Entrena y evalua el modelo. Usa name='mlp_shuffle'
train_loss_df, val_loss_df, test_loss, test_accuracy =




# Guarda el modelo. Usa file_name='mlp_shuffle.pth'


# Carga los datos y muéstralos
train_loss_file = 'train_loss_mlp_shuffle.csv'  
valid_loss_file = 'valid_loss_mlp_shuffle.csv'
train_accuracy_file = 'train_accuracy_mlp_shuffle.csv'
valid_accuracy_file = 'valid_accuracy_mlp_shuffle.csv'

plot_loss_accuracy(train_loss_file, valid_loss_file, train_accuracy_file, valid_accuracy_file)

# Calcula la matriz de confusión
cm = plot_confusion_matrix(model_mlp, test_loader_shuffle, device=device)

# Muestra el porcentaje de error por clase
plot_error_per_class(cm)


### Entrena la CNN con:

* `train_loader_shuffle`
* `valid_loader_shuffle`
* `test_loader_shuffle`

Guarda el modelo como `cnn_shuffle.pth` y los resultados de `loss`y `accuracy` con la misma extensión:

In [ ]:
# Instancia el modelo
model_cnn = 

# Instancia la función de pérdida y el optimizador
loss_fn = 
optimizer = 

# Fija 10 épocas
num_epochs = 10

# Entrena y evalua el modelo. Usa name='cnn_shuffle'
train_loss_df, val_loss_df, test_loss, test_accuracy = 




# Guarda el modelo. Usa file_name='cnn_shuffle.pth'


# Carga los datos y muéstralos
train_loss_file = 'train_loss_cnn_shuffle.csv'  
valid_loss_file = 'valid_loss_cnn_shuffle.csv'
train_accuracy_file = 'train_accuracy_cnn_shuffle.csv'
valid_accuracy_file = 'valid_accuracy_cnn_shuffle.csv'

plot_loss_accuracy(train_loss_file, valid_loss_file, train_accuracy_file, valid_accuracy_file)

# Calcula la matriz de confusión
cm = plot_confusion_matrix(model_cnn, test_loader_shuffle, device=device)

# Muestra el porcentaje de error por clase
plot_error_per_class(cm)

¿Cómo podrías explicar los resultados?

############## COMPLETAR ##############

############ Fin COMPLETAR ############

## **[Ejercicio 3]** Entrena una `FCNN` con las siguientes características:

1. Cuatro bloques convolucionales (conv2d -> batch norm -> ReLU).
2. Un bloque convolucional final con tantas características como clases haya (43)
3. Comprueba el número de parámetros entrenables y busca una configuración que no supere los **50.000** parámetros (aproximadamente 4 veces menos que la MLP).

In [ ]:
class FCNN(nn.Module):
    def __init__(self, output_dim):
        super(FCNN, self).__init__()

        # Primer bloque convolucional

        # Segundo bloque convolucional
        
        # Tercer bloque convolucional

        # Cuarto bloque convolucional

        # Bloque de salida

    def forward(self, x):
        # Primer bloque convolucional
     

        # Segundo bloque convolucional
    

        # Tercer bloque convolucional
      

        # Cuarto bloque convolucional
    

        # Capa de salida
        x =   # Salida (N, num_classes, 1, 1)
        x =   # (N, num_classes)
        return x

count_trainable_params(FCNN(output_dim=43))

Añade el modelo a tu archivo `models.py`.

### Entrena la FCNN con el dataset original (sin permutar)

* `train_loader`
* `valid_loader`
* `test_loader`

Guarda el modelo como `fcnn.pth` y los resultados de `loss`y `accuracy` con la misma extensión:

In [ ]:
# Instancia el modelo
model_fcnn = 

# Instancia la función de pérdida y el optimizador
loss_fn = 
optimizer = 

# Fija 10 épocas
num_epochs = 10

# Entrena y evalua el modelo. Usa name='fcnn'
train_loss_df, val_loss_df, test_loss, test_accuracy =




# Guarda el modelo. Usa file_name='fcnn.pth'


# Carga los datos y muéstralos
train_loss_file = 'train_loss_fcnn.csv'  
valid_loss_file = 'valid_loss_fcnn.csv'
train_accuracy_file = 'train_accuracy_fcnn.csv'
valid_accuracy_file = 'valid_accuracy_fcnn.csv'

plot_loss_accuracy(train_loss_file, valid_loss_file, train_accuracy_file, valid_accuracy_file)

# Calcula la matriz de confusión
cm = plot_confusion_matrix(model_fcnn, test_loader, device=device)

# Muestra el porcentaje de error por clase
plot_error_per_class(cm)

# **[Ejercicio 4]** Entrena una MLP y una FCNN con el dataset modificado

1. Crea un modelo con las características que quieras siempre y cuando no supere los **500.000** parámetros.
2. Entrena **30** épocas **SIN** aumentado de datos

In [ ]:
import random
from PIL import Image
import torchvision.transforms as T

import random
import hashlib
from PIL import Image

class DeterministicResizeAndPaste:
    def __init__(self, canvas_size, scale_range=(0.7, 0.9)):

        self.canvas_size = canvas_size if isinstance(canvas_size, tuple) else (canvas_size, canvas_size)
        self.scale_range = scale_range

    def _get_image_seed(self, img):

        img_bytes = img.tobytes()
        hash_object = hashlib.md5(img_bytes)
        return int(hash_object.hexdigest(), 16)

    def __call__(self, img):
        seed = self._get_image_seed(img)
        rng = random.Random(seed) 
        scale = rng.uniform(self.scale_range[0], self.scale_range[1])        

        orig_w, orig_h = img.size
        new_w = int(orig_w * scale)
        new_h = int(orig_h * scale)
        
        img_resized = img.resize((new_w, new_h), Image.BILINEAR)        
        canvas = Image.new('RGB', self.canvas_size, (0, 0, 0))
        
        max_x = max(0, self.canvas_size[1] - new_w)
        max_y = max(0, self.canvas_size[0] - new_h)
        
        pos_x = rng.randint(0, max_x)
        pos_y = rng.randint(0, max_y)
        
        canvas.paste(img_resized, (pos_x, pos_y))        
        return canvas

# Definir las transformaciones 
transform = transforms.Compose([
    DeterministicResizeAndPaste(canvas_size=32),
    transforms.ToTensor()
])
train_dataset_ds2 = GTS(csv_file='train.csv', root_dir='./data', transform=transform)
valid_dataset_ds2 = GTS(csv_file='valid.csv', root_dir='./data', transform=transform)
test_dataset_ds2 = GTS(csv_file='test.csv', root_dir='./data', transform=transform)

device = 'cpu'
batch_size = 16

train_loader_ds2 = DataLoader(train_dataset_ds2, batch_size=batch_size, shuffle=True, num_workers=4)
valid_loader_ds2 = DataLoader(valid_dataset_ds2, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader_ds2 = DataLoader(test_dataset_ds2, batch_size=batch_size, shuffle=False, num_workers=4)


# Visualizar ejemplos del conjunto de entrenamiento
visualize_subset(train_dataset_ds2)